# 0. Imports and loading data

In [ ]:
import re
from pathlib import Path
import inspect
from typing import Any

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

from scipy.sparse import csr_matrix

from wordcloud import WordCloud
from tqdm import tqdm

from nltk import download as nltk_download
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

import spacy

!python -m spacy download en_core_web_sm

# Download necessary NLTK resources
nltk_download("punkt")
nltk_download("stopwords")
nltk_download("wordnet")
nltk_download("averaged_perceptron_tagger")
nltk_download("omw-1.4")

In [ ]:
# Define project directories
BASE_DIR: Path = Path().resolve().parent
DATA_DIR: Path = BASE_DIR / "data"

# Load data
df: pd.DataFrame = pd.read_csv(
    DATA_DIR / "intermediate" / "25072024_reviews_openrefine.csv"
)

df.head()

# 1. Processing text: comparing spaCy and NLTK

Text data must be processed before it can be used in machine learning models. This processing includes tokenization, lemmatization, and removing stopwords and punctuation. Here we will try out two popular libraries for text processing: spaCy and NLTK.

## 1.1. NLTK

NLTK (natural language toolkit) is an extensive library for a wide range of NLP tasks. Its the older of the two, first released in 2001. It contains a lot of functionality, but requires more manual work to get started.

Use t-SNE for visualization (and try different parameters to get something visually pleasing!), but rather do not run clustering afterwards, in particular do not use distance- or density based algorithms, as this information was intentionally (!) lost. Neighborhood-graph based approaches may be fine, but then you don't need to first run t-SNE beforehand, just use the neighbors immediately (because t-SNE tries to keep this nn-graph largely intact).

In [ ]:
def prepare_text(text: str, processor: WordNetLemmatizer | PorterStemmer) -> str:
    """Preparing text for TF-IDF vectorization."""

    # Convert to lowercase and remove non-alphabetic characters
    processed_text: str = re.sub(r"[^a-z]", " ", text.lower())

    # Tokenize the text
    tokens: list[str] = word_tokenize(processed_text)

    # POS tagging. Nltk POS tags are based solely on token, without taking into account the context.
    tagged_tokens: list[tuple[Any, str]] = pos_tag(tokens)

    # Apply the processor (lemmatizer or stemmer). Lemmatizer is better than stemmer.
    # get_wordnet_tag to map the treebank pos tags to wordnet tags
    if isinstance(processor, WordNetLemmatizer):
        processed_tokens = [
            processor.lemmatize(token, pos=get_wordnet_tag(tag))
            for token, tag in tagged_tokens
        ]
    elif isinstance(processor, PorterStemmer):
        processed_tokens = [processor.stem(token) for token in tokens]
    else:
        raise ValueError(
            "Processor must be an instance of either WordNetLemmatizer or PorterStemmer"
        )
    # Join the processed tokens back into a single string and return
    return " ".join(processed_tokens)


def get_wordnet_tag(treebank_tag) -> str:
    """
    Transform treebank pos tags to wordnet tags for WordNet lemmatizer.
    """
    if treebank_tag.startswith("J"):
        return wordnet.ADJ
    elif treebank_tag.startswith("V"):
        return wordnet.VERB
    elif treebank_tag.startswith("N"):
        return wordnet.NOUN
    elif treebank_tag.startswith("R"):
        return wordnet.ADV
    else:
        # As default pos in lemmatization is Noun
        return wordnet.NOUN

In [ ]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Apply the text processing function to the "blind_assessment" column
prepared_text: pd.Series = df["blind_assessment"].apply(
    prepare_text, processor=lemmatizer
)

In [ ]:
# Apply TF-IDF to the prepared text
# ==================================================#

vectorizer: TfidfVectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix: csr_matrix = vectorizer.fit_transform(
    prepared_text
)  # -> scipy sparse matrix

feature_names: list[str] = vectorizer.get_feature_names_out().tolist()
tfidf_df: pd.DataFrame = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

print(tfidf_matrix.toarray().shape)

# Check that the number of rows matches
assert tfidf_matrix.shape[0] == df.shape[0]

tfidf_df.head()

In [ ]:
# Create a wordcloud of top keywords
# ================================== #
def plot_wordcloud(
    word_scores: dict[str, float], fout: Path, title: str | None = None, **kwargs
):
    """Plotting a wordcloud from a dictionary of word scores."""

    valid_args = inspect.signature(WordCloud).parameters
    for key in kwargs:
        if key not in valid_args:
            raise ValueError(f"Invalid keyword argument: {key}")

    wordcloud = WordCloud(**kwargs)
    wordcloud.generate_from_frequencies(word_scores)

    plt.figure(figsize=(20, 10), dpi=300)
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    if title:
        plt.title(title, fontsize=20)
    plt.savefig(fout, bbox_inches="tight")
    plt.show()


output_path: Path = BASE_DIR / "imgs" / "blind_assessment_tfidf_wordcloud.png"
tfidf_scores: np.ndarray = tfidf_matrix.mean(axis=0).A1

word_scores: dict[str, float] = dict(zip(feature_names, tfidf_scores))

plot_wordcloud(
    word_scores,
    fout=output_path,
    width=1600,
    height=800,
    background_color="white",
    max_font_size=200,
    max_words=300,
)

## 1.2 spaCy

In [ ]:
nlp = spacy.load("en_core_web_sm")

# Apply spacy to prepared_text
# ============================ #

docs = list(nlp.pipe(df["blind_assessment"].tolist()))

In [ ]:
prepared_text = []

for doc in docs:
    joined_tokens = " ".join(
        [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    )
    prepared_text.append(joined_tokens.lower())


# Checking we didnt lose any rows
assert len(prepared_text) == df.shape[0]

In [ ]:
prepared_text = pd.Series(prepared_text)

# Apply TF-IDF vectorization to cleaned_assessment column
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(prepared_text)  # -> scipy sparse matrix
feature_names = vectorizer.get_feature_names_out()


tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)


print(tfidf_matrix.toarray().shape)
# Check that the number of rows match
assert tfidf_matrix.shape[0] == df.shape[0]

tfidf_df.head()

In [ ]:
output_path = BASE_DIR / "imgs" / "blind_assessment_tfidf_wordcloud_spacy.png"
tfidf_scores = tfidf_matrix.mean(axis=0).A1
word_scores: dict[str, float] = dict(zip(feature_names, tfidf_scores))

plot_wordcloud(
    word_scores,
    fout=output_path,
    width=1600,
    height=800,
    background_color="white",
    max_font_size=200,
    max_words=300,
)

## Clustering

We will two clustering algorithms: Kmeans and DBSCAN. Each will run on the original tf-ifd vectors and on PCA reduced vectors.

Clusters will be visualized in 2D using t-SNE. Note that t-SNE should not be used for dimensionality reduction and only for producing visualizations.

### PCA

Dimensionality reduction can be helpful for clustering methods as they are typically sensitive to the *curse of dimensionality*.


### Kmeans

Kmeans is a simple clustering algorithm that partitions the data into k clusters where k must be specified. To determine the optimal number of clusters, we will use the elbow method.
This method consists of plotting the sum of squared distances between each point and the centroid of the cluster it belongs to. The optimal number of clusters is the point where the curve starts to flatten out. With the optimal k value, we will run Kmeans on the original tf-idf vectors and on PCA reduced data. 

### DBSCAN

DBSCAN is a density-based clustering algorithm that groups together points that are closely packed together. It is able to find clusters of arbitrary shapes and sizes. DBSCAN has two parameters: epsilon and min_samples. Epsilon is the maximum distance between two samples for one to be considered as in the neighborhood of the other. Min_samples is the number of samples in a neighborhood for a point to be considered as a core point. Epsilon is the most important parameter to tune. Performs DBSCAN over varying epsilon values and integrates the result to find a clustering that gives the best stability over epsilon. This allows HDBSCAN to find clusters of varying densities (unlike DBSCAN), and be more robust to parameter selection

### HDBSCAN

HDBSCAN is a hierarchical version of DBSCAN. It is able to find clusters of varying densities and sizes. It has two parameters: min_cluster_size and min_samples. Min_cluster_size is the smallest size grouping that should be considered a cluster. Min_samples is the number of samples in a neighborhood for a point to be considered as a core point. HDBSCAN is much more than scale invariant though – it is capable of multi-scale clustering, which accounts for clusters with varying density. Traditional DBSCAN assumes that any potential clusters are homogeneous in density. HDBSCAN is free from such constraints.

### t-SNE
t-SNE is a manifold learning method for visualizing high-dimensional data. It maps the high-dimensional data to a 2D or 3D space so that the data points that are close in the high-dimensional space remain close in the low-dimensional space. t-SNE is generally not recommended for dimensionality reduction as it does not preserve global structure and distances.

https://distill.pub/2016/misread-tsne/


### UMAP

Notes on umap, tf-ifd for text analysiss and visulization

https://umap-learn.readthedocs.io/en/latest/sparse.html

### Can I cluster the results of UMAP?

This is hard to answer well, but essentially the answer is “yes, with care”. To start with it matters what clustering algorithm you are going to use. Since UMAP does not necessarily produce clean spherical clusters something like K-Means is a poor choice. I would recommend HDBSCAN or similar. The catch here is that UMAP, with its uniform density assumption, does not preserve density well. What UMAP will do, however, is contract connected components of the manifold together. Providing you have enough data for UMAP to distinguish that information then you can get useful clustering results out since algorithms like HDBSCAN will easily pick out the components after applying UMAP.

UMAP does offer significant improvements over algorithms like t-SNE for clustering. First, by preserving more global structure and creating meaningful separation between connected components of the manifold on which the data lies, UMAP offers more meaningful clusters. Second, because it supports arbitrary embedding dimensions, UMAP allows embedding to larger dimensional spaces that make it more amenable to clustering.

Consider a typical pipeline: high-dimensional embedding (300+) => PCA to reduce to 50 dimensions => UMAP to reduce to 10-20 dimensions => HDBSCAN for clustering / some plain algorithm for classification;

data -> pre-process -> tf-idf vectors

Then... 

Kmeans (silhouette scores) -> t-SNE, UMAP

PCA -> Kmeans (silhouette scores) -> t-SNE, UMAP

PCA -> HDBSCAN -> t-SNE, UMAP

PCA -> UMAP -> HDBSCAN -> t-SNE, UMAP 


In [ ]:
# df - read in data
# tfidf_matrix - matrix of tfidf values
# vectorizer - object returned by TfidfVectorizer that contains the feature names and scores

# Kmeans clustering
X = tfidf_matrix.toarray()

pca = PCA()
X_pca = pca.fit_transform(X)

# plot explained variance ratiio

plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel("number of components")
plt.ylabel("cumulative explained variance")

In [ ]:
# keep 95% of variance
n_components = np.argmax(np.cumsum(pca.explained_variance_ratio_) > 0.95)
print(f"Number of components to keep: {n_components}")

# keep the first n_components from our pca
X_pca = X_pca[:, :n_components]
X_pca.shape

In [ ]:
# Use silhouette score to determine the number of clusters
silhouette_scores = []
K = range(30, 100)

for k in tqdm(K):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_pca)
    cluster_labels = kmeans.predict(X_pca)
    silhouette = silhouette_score(X_pca, cluster_labels)
    silhouette_scores.append(silhouette)

# number of clusters with the highest silhouette score
best_k = K[np.argmax(silhouette_scores)]

In [ ]:
px.line(
    x=K,
    y=silhouette_scores,
    height=400,
    width=800,
    labels={"x": "Number of Clusters", "y": "Silhouette Score"},
)

In [ ]:
# number of clusters with the highest silhouette score
best_k = K[np.argmax(silhouette_scores)]

kmeanModel = KMeans(n_clusters=best_k).fit(X_pca)
cluster_labels = kmeanModel.predict(X_pca)

In [ ]:
# Perform t-SNE on the PCA features to visualize the clusters
tsne = TSNE(n_components=2, perplexity=50, n_iter=5000, random_state=42)
tsne_features = tsne.fit_transform(X_pca)

In [ ]:
tsne_df = pd.DataFrame(tsne_features, columns=["component1", "component2"])

tsne_df["cluster"] = cluster_labels
tsne_df["cluster"] = tsne_df["cluster"].astype(str)
tsne_df["country"] = df["origin_country"]
tsne_df["rating"] = df["rating"]

fig = px.scatter(
    tsne_df,
    x="component1",
    y="component2",
    color="cluster",
    # hover shows country and rating
    hover_data={"cluster": True, "country": True, "rating": True},
    height=800,
    width=1200,
)

fig.show()

In [ ]:
# Kmeans
# Kmeans after PCA
# DBSCAN - fine tune the hyperparameters
# DBSCAN with cosine metric

# Plot the clusters using t-SNE, and fine-tuning

## Dimensionality Reduction and Visualization
t-SNE
PCA